# 性能衡量和提升
- 衡量代码的性能。
- 一些提高代码性能的技巧。
- cv.getTickCount，**cv.getTickFrequency**等。

除了OpenCV，Python还提供了一个模块**time**，这有助于衡量执行时间。另一个模块**profile**有助于获取有关代码的详细报告，例如代码中每个函数花费了多少时间，调用了函数的次数等。

In [1]:
import numpy as np
import cv2 as cv

## 使用OpenCV衡量性能
**cv.getTickCount**函数返回从参考事件（如打开机器的那一刻）到调用此函数那一刻之间的时钟周期数。因此，如果在函数执行之前和之后调用它，则会获得用于执行函数的时钟周期数。

**cv.getTickFrequency**函数返回时钟周期的频率或每秒的时钟周期数。因此，要找到执行时间（以秒为单位），你可以执行以下操作：

In [2]:
e1 = cv.getTickCount()
#执行代码
e2 = cv.getTickCount()
time = (e2 - e1) / cv.getTickFrequency()
print(time)

5.35e-05


下面的示例应用中位数过滤，其内核的奇数范围为5到49

In [3]:
img1 = cv.imread('./images/messi1.png')
e1 = cv.getTickCount()
for i in range(5, 49, 2):
    img1 = cv.medianBlur(img1, i) # 使用中值滤波器模糊图像。该函数使用具有 ksize×ksize 孔径的中值滤波器平滑图像。 多通道图像的每个通道都是
                                # 独立处理的。 支持就地操作
e2 = cv.getTickCount()
t = (e2 - e1) / cv.getTickFrequency()
print( t )

1.7013307


你可以使用时间模块执行相同的操作。代替cv.getTickCount，使用time.time()函数。然后取两次相差。

## OpenCV中的默认优化
许多 OpenCV 函数都是使用 SSE2、 AVX 等进行优化的。 它还包含未优化的代码。因此，如果我们的系统支持这些特性，我们就应该利用它们(几乎所有现代的处理器都支持它们)。在编译时默认启用它。因此，如果启用了 OpenCV，它将运行优化的代码，否则它将运行未优化的代码。你可以使用 cvUseoptimized 检查是否启用 / 禁用和 cvSetuseoptimized 以启用 / 禁用它。让我们看一个简单的例子。
检查是否启用了优化

In [4]:
cv.useOptimized()

True

In [6]:
%timeit res = cv.medianBlur(img1 ,49)

68.4 ms ± 3.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
cv.setUseOptimized(False)
cv.useOptimized()

False

In [8]:
%timeit res = cv.medianBlur(img1 ,49)

73.2 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


你可以看到中值滤波是 SIMD 优化。因此，你可以使用它在代码顶部启用优化(请记住，它是默认启用的)

In [9]:
cv.setUseOptimized(True)
cv.useOptimized()

True

## 在IPython中衡量性能
有时你可能需要比较两个类似操作的性能。IPython为你提供了一个神奇的命令计时器来执行此操作。它会多次运行代码以获得更准确的结果。同样，它们适用于测量单行代码。

例如，你知道以下哪个加法运算更好，`x = 5; y = x**2, x = 5; y = x*x, x = np.uint8([5]); y = x*x`或`y = np.square(x)`?我们将在IPython shell中使用timeit得到答案

In [10]:
x = 5

IPython魔术命令以百分号%为前缀；可看作运行于IPython系统中的命令行程序，它们大都还有一些参数选项。在命令后面加问号(?)可以查看。默认可以不带百分号使用的，只要没有定义与其同名的变量即可。可以通过%automagic命令打开或者关闭此功能。

%timeit
多次执行一条语句，并返回平均时间，

%%timeit
多次执行多条语句，并返回平均时间，

%time
返回执行一条语句的时间，

%%time
返回执行多条语句的时间，

%reset
删除当前空间的全部变量

%run *.py
在IPython中执行Python脚本

魔术命令+(?)显示文档
如:%time?

常用的魔术命令如下：

%quickref thon 快速参考

%magic 显示magic command详细文档

%debug 从最新的异常跟踪的底部进入交互式调试器

%hist 打印命令输入历史

%pdb 在发生异常后自动进入调试器

%paste执行剪贴板中的Python代码

%cpaste 打开一个特殊的提示符以便手工粘贴待执行的代码

%reset 删除interactive空间中的全部变量/名称

%run 执行一个python脚本

%page 分页显示一个对象

%time 报告statement执行的时间

%timeit 多次执行statement以计算平均执行时间，用于执行时间非常小的代码。

%who、%who_is、%whos 显示Interactive命名空间的中定义的变量，信息级别/冗余度可变

%xdel 删除变量，并尝试清楚其在IPython中的对象上的一切引用

In [12]:
%timeit y=x**2 

274 ns ± 49.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
%timeit y = x*x

52.2 ns ± 3.24 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [14]:
z = np.uint8([5])

In [15]:
%timeit y=z*z

411 ns ± 17.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [16]:
%timeit y=np.square(z)

412 ns ± 25.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


你可以看到x = 5; y = x * x最快，比Numpy快20倍左右。如果你还考虑阵列的创建，它可能会快100倍。cool?
（大量开发人员正在研究此问题）

注意 Python标量操作比Numpy标量操作快。因此，对于包含一两个元素的运算，Python标量比Numpy数组好。当数组大小稍大时，Numpy会占优势。

我们将再尝试一个示例。这次，我们将比较**cv.countNonZero**和**np.count_nonzero**对于同一张图片的性能。

In [21]:
a=cv.countNonZero(img1) #src shuold be single-channel array. 所以找张灰度图吧 我懒得找了qwq

error: OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\core\src\count_non_zero.dispatch.cpp:128: error: (-215:Assertion failed) cn == 1 in function 'cv::countNonZero'


In [20]:
%timeit b=np.count_nonzero(img1)

146 µs ± 9.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


注意 通常，OpenCV函数比Numpy函数要快。因此，对于相同的操作，首选OpenCV功能。但是，可能会有例外，尤其是当Numpy处理视图而不是副本时

## 性能优化技术
有几种技术和编码方法可以充分利用 Python 和 Numpy 的最大性能。这里只注明相关信息，并提供重要信息来源的链接。这里要注意的主要事情是，首先尝试以一种简单的方式实现算法。一旦它运行起来，分析它，找到瓶颈并优化它们。

1. 尽量避免在Python中使用循环，尤其是双/三重循环等。它们本来就很慢。
2. 由于Numpy和OpenCV已针对向量运算进行了优化，因此将算法/代码向量化到最大程度。
3. 利用缓存一致性。
4. 除非需要，否则切勿创建数组的副本。尝试改用视图。数组复制是一项昂贵的操作

即使执行了所有这些操作后，如果你的代码仍然很慢，或者不可避免地需要使用大循环，请使用Cython等其他库来使其更快。